In [1]:
import cooler
import bioframe as bf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import bbi
from matplotlib import gridspec
from tensorflow import keras
import tensorflow.keras.backend as K
import tensorflow as tf
import regionsLoader

In [2]:
regions_path = '/home/users/luisfcd/data/regions/regions_10kb.tsv'
cool_path = '/home/users/luisfcd/data/Hi-C/GM12878/4DNFIXP4QG5B.mcool'
chip_path = '/home/users/luisfcd/data/CHIP/GM12878/'
regionsize = 4000000
resolution=10000

In [3]:
bigwig_dict = {}
for path in glob(chip_path+'**/*.bigWig', recursive=True):
    bigwig_dict[path.split('/')[-2]]=path

bigwig_dict

{'H3K27ac': '/home/users/luisfcd/data/CHIP/GM12878/H3K27ac/ENCFF469WVA.bigWig',
 'H3K27me3': '/home/users/luisfcd/data/CHIP/GM12878/H3K27me3/ENCFF919DOR.bigWig',
 'H3K9me3': '/home/users/luisfcd/data/CHIP/GM12878/H3K9me3/ENCFF683HCZ.bigWig'}

In [4]:
regions_array = np.squeeze(pd.read_csv(regions_path, header=None).values)
regions_df = bf.from_any(list(regions_array))

In [5]:
def get_all_regions(regions_df, regionsize, step=10000):
    mask = (regions_df.end - regions_df.start) > regionsize
    all_regions = []
    for i, row in regions_df[mask].iterrows():
        for k in range(0, (row.end - row.start)-regionsize+1, step):
            region = bf.to_ucsc_string((row.chrom, row.start+k, row.start+k+regionsize))
            all_regions.append(region)
    
    return bf.from_any(all_regions)

In [6]:
all_regions_df = get_all_regions(regions_df, regionsize)

In [7]:
all_regions_df.iloc[[1,2, 10]]

,chrom,start,end
1,chr1,2791250,6791250
2,chr1,2801250,6801250
10,chr1,2881250,6881250


In [8]:
target_regions_df = bf.select(all_regions_df, 'chr5').sample(frac=1, random_state=28) #select a chromosome and shuffle it
train_regions_df = target_regions_df.iloc[:int(len(target_regions_df)*0.8)].reset_index(drop=True)
test_regions_df = target_regions_df.iloc[int(len(target_regions_df)*0.8):].reset_index(drop=True)

In [9]:
chroms = all_regions_df.chrom.unique()[1:4]
for chrom in chroms:
    target_regions_df = bf.select(all_regions_df, chrom).sample(frac=1, random_state=28)
    train_regions_df_temp = target_regions_df.iloc[:int(len(target_regions_df)*0.8)].reset_index(drop=True)
    train_regions_df = pd.concat((train_regions_df, train_regions_df_temp), ignore_index=True)

In [10]:
def build_set(hic_generator, chip_generator_dict, batchsize=1000):
    targets = []
    inputs = []
    for element in range(batchsize):
        mat = next(hic_generator)
        targets.append(mat)
        signal_list = list()
        for k, (key, chip_generator) in enumerate(chip_generator_dict.items()):
            signal_list.append(next(chip_generator))
            
        inputs.append(np.array(signal_list).T)
    
    return np.array(inputs), np.array(targets)

In [11]:
hic_generator = regionsLoader.hic_loader(train_regions_df, cool_path, resolution=50000)
chip_generator_dict = {}

for key, path in bigwig_dict.items():
    print(key)
    chip_generator_dict[key] = regionsLoader.chip_loader(train_regions_df, path, resolution=10000)

train_inputs, train_targets = build_set(hic_generator, chip_generator_dict, batchsize=20)

H3K27ac
H3K27me3
H3K9me3


In [12]:
np.pad(train_targets[0, :80-2,:80-2], 1, mode='edge').shape

(80, 80)

In [13]:
hic_generator = regionsLoader.hic_loader(test_regions_df, cool_path, resolution=50000)
chip_generator_dict = {}

for key, path in bigwig_dict.items():
    print(key)
    chip_generator_dict[key] = regionsLoader.chip_loader(test_regions_df, path, resolution=10000)

test_inputs, test_targets = build_set(hic_generator, chip_generator_dict, batchsize=20)

H3K27ac
H3K27me3
H3K9me3


In [14]:
test_targets.shape

(20, 80, 80)

In [15]:
N=80

In [16]:
# @tf.function
def flat_to_mat_TF(vector, n=N):
    idx = list(zip(*np.triu_indices(n)))
    idx = tf.constant([list(i) for i in idx], dtype=tf.int64)
    sp_input = tf.SparseTensor(indices = idx, values = vector, dense_shape=(n,n))
        
    mat = tf.sparse.to_dense(sp_input, default_value=0)
    mat = mat + K.transpose(mat)
    
    mat = tf.linalg.set_diag(mat, tf.linalg.diag_part(mat)/2)
    return mat

In [17]:
idx = np.arange(0, N)[:, None]
distance = np.abs(idx-idx.T)+0.1
iu = np.triu_indices(N)
distance_iu = distance[iu]

In [18]:
def create_model(model_input, N):
    
    x = keras.layers.Conv1D(name='Conv1a', filters = 32, kernel_size= 5, dilation_rate=2, 
                            padding='same', activation='relu')(model_input)
    x = keras.layers.Conv1D(name='Conv1b', filters = 32, kernel_size= 5, dilation_rate=2, 
                            padding='same', activation='relu')(x)
    x = keras.layers.MaxPool1D(name='Pool1', pool_size=2)(x)
    
    
    
    x = keras.layers.Conv1D(name='Conv2a', filters = 64, kernel_size= 5, dilation_rate=4,
                            padding='same', activation='relu')(x)
    x = keras.layers.Conv1D(name='Conv2b', filters = 64, kernel_size= 5, dilation_rate=4,
                            padding='same', activation='relu')(x)
    x = keras.layers.MaxPool1D(name='Pool2', pool_size=2)(x)
    
    
    x = keras.layers.Conv1D(name='Conv3a', filters = 64, kernel_size= 5, dilation_rate=2,
                        padding='valid', activation='relu')(x)
    x = keras.layers.Conv1D(name='Conv3b', filters = 64, kernel_size= 5, dilation_rate=2,
                        padding='valid', activation='relu')(x)
    
    
    x = keras.layers.Conv1D(name='Conv3c', filters = 128, kernel_size= 3, dilation_rate=2,
                    padding='valid', activation='relu')(x)
    
    
    x = keras.layers.Conv1D(name='Conv3d', filters = 64, kernel_size= 3,
                padding='same', activation='linear')(x)
    
    
    x = keras.layers.Lambda(lambda x: K.expand_dims(x, axis=1), name='expand_dims')(x)
    xt = keras.layers.Permute( (2, 1, 3), name='permute_0')(x)
    x = keras.layers.Add(name='add_0')([x, xt])
    

    x_in = x
    x = keras.layers.Conv2D(64, 3, activation='relu', padding='same',name='Conv2Da')(x)
    x = keras.layers.Conv2D(128, 1, activation='relu', padding='same',name='Conv2Db')(x)
    x = keras.layers.Conv2D(64, 3, activation='linear', padding='same', name='Conv2Dc')(x)
    xt = keras.layers.Permute( (2, 1, 3), name='permute')(x)
    x = keras.layers.Add(name='add')([x, xt, x_in])
    
    
    x_in = x
    x = keras.layers.Conv2D(64, 3, activation='relu', padding='same',name='Conv2Da_2')(x)
    x = keras.layers.Conv2D(128, 1, activation='relu', padding='same',name='Conv2Db_2')(x)
    x = keras.layers.Conv2D(64, 3, activation='linear', padding='same', name='Conv2Dc_2')(x)
    xt = keras.layers.Permute( (2, 1, 3), name='permute_2')(x)
    x = keras.layers.Add(name='add_2')([x, xt, x_in])
    
    
    x_in = x
    x = keras.layers.Conv2D(64, 3, activation='relu', padding='same',name='Conv2Da_3')(x)
    x = keras.layers.Conv2D(128, 1, activation='relu', padding='same',name='Conv2Db_3')(x)
    x = keras.layers.Conv2D(64, 3, activation='linear', padding='same', name='Conv2Dc_3')(x)
    xt = keras.layers.Permute( (2, 1, 3), name='permute_3')(x)
    x = keras.layers.Add(name='add_3')([x, xt, x_in])
    
    
#     x = keras.layers.Dense(256, activation='relu',name='Dense_semifinal1')(x)
#     x = keras.layers.Dense(512, activation='relu',name='Dense_semifinal2')(x)

    x = keras.layers.Dense(1, name='Dense_final')(x)
    
    x = regionsLoader.layers.UpperTri(diagonal_offset=0)(x)
    
    
    
    
    

    x = keras.layers.Flatten(name='Flatten')(x)
#     x = keras.layers.Dense(512, activation='relu', name='Dense_1')(x)
#     x = keras.layers.Dense(((N*N)//2) + (N//2), name='Dense_final')(x)
#                            kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
#                            bias_regularizer=keras.regularizers.l2(1e-4))(x),
#     x = keras.layers.Reshape((80,80), name='Reshaper')(x)
    x = keras.backend.exp(x)
    return x

In [19]:
@tf.function
def custom_loss(y_actual,y_pred):
#     print('y_actual.shape', y_actual.shape)
#     y_pred = tf.map_fn(flat_to_mat_TF, y_pred)
    y_actual = K.log(y_actual+1e-10)
    y_pred = K.log(y_pred+1e-10)
    loss_square = K.square(y_actual-y_pred)*distance_iu
    
    custom_loss = tf.reduce_sum(loss_square, axis=-1)
    custom_loss = tf.reduce_mean(custom_loss, axis=-1)
    
    return custom_loss

In [20]:
model_input = keras.Input(shape=(400, 3, ), name='model_input')
model = keras.Model(inputs=model_input, outputs=create_model(model_input, N), name='model_1')

model.compile(optimizer='adam',
              loss=custom_loss, run_eagerly=False,)

2022-04-05 14:29:07.021169: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-05 14:29:07.021475: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-05 14:29:07.104507: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [21]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
model_input (InputLayer)        [(None, 400, 3)]     0                                            
__________________________________________________________________________________________________
Conv1a (Conv1D)                 (None, 400, 32)      512         model_input[0][0]                
__________________________________________________________________________________________________
Conv1b (Conv1D)                 (None, 400, 32)      5152        Conv1a[0][0]                     
__________________________________________________________________________________________________
Pool1 (MaxPooling1D)            (None, 200, 32)      0           Conv1b[0][0]                     
____________________________________________________________________________________________

In [22]:
# K.set_value(model.optimizer.learning_rate, 1e-3)

In [23]:
train_regions_df = train_regions_df.sample(frac=1, random_state=99).reset_index(drop=True)

In [24]:
train_regions_df

,chrom,start,end
0,chr4,44303750,48303750
1,chr2,188313750,192313750
2,chr2,35991250,39991250
3,chr2,168665000,172665000
4,chr2,114400000,118400000
...,...,...,...
47283,chr4,70575000,74575000
47284,chr4,149535000,153535000
47285,chr4,21950000,25950000
47286,chr3,30267500,34267500


In [25]:
from importlib import reload
reload(regionsLoader)
reload(regionsLoader.layers)
reload(regionsLoader.generator)
chip_names =['H3K27ac', 'H3K27me3', 'H3K9me3']
train_generator = regionsLoader.generator.DataGenerator(train_regions_df, chip_names, 
                            cool_path=cool_path, chip_path_dict = bigwig_dict, 
                            cool_resolution=50000, chip_resolution=10000, batch_size=16)

test_generator = regionsLoader.generator.DataGenerator(test_regions_df, chip_names, 
                            cool_path=cool_path, chip_path_dict = bigwig_dict, 
                            cool_resolution=50000, chip_resolution=10000, batch_size=16)

In [26]:
model=keras.models.load_model('conv_model2.h5', 
                              custom_objects={'UpperTri': regionsLoader.layers.UpperTri,
                                             'custom_loss': custom_loss})

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [27]:
K.set_value(model.optimizer.learning_rate, 1e-4)

In [ ]:
history = model.fit(train_generator, epochs=2)

2022-04-05 14:29:09.500625: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-04-05 14:29:09.503918: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2095050000 Hz


Epoch 1/2
 858/2955 [=======>......................] - ETA: 1:11:46 - loss: 17458.3066

In [ ]:
# model.save('conv_model2.h5')

In [ ]:
plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
test_predictions = model.predict(test_inputs)
train_predictions = model.predict(train_inputs)

In [ ]:
def plot_predictions(targets, predictions):
    fig = plt.figure(figsize= (16, 16))
    for i in range(8):
        ax = plt.subplot(4,4, (i*2)+1)
        ax.matshow(np.log(targets[i]+5e-6), cmap='YlOrRd')
        ax.set_title('target')
        ax = plt.subplot(4,4, (i*2)+2)
        ax.set_title('prediction')
        ax.matshow(np.log(regionsLoader.from_upper_triu(predictions[i], N)+5e-6), cmap='YlOrRd', 
                   vmax=np.nanmax(np.log(targets[i]+5e-6)), 
                   vmin=np.nanmin(np.log(targets[i]+5e-6)))

In [ ]:
plot_predictions(train_targets, train_predictions)

In [ ]:
plot_predictions(test_targets, test_predictions)

In [ ]:
chromsizes = bf.fetch_chromsizes('hg38').iloc[11:12]
region_chrom = bf.binnify(chromsizes, binsize=4000000)[:-1]

In [ ]:
hic_generator = regionsLoader.hic_loader(region_chrom, cool_path, resolution=50000)
chip_generator_dict = {}

for key, path in bigwig_dict.items():
    print(key)
    chip_generator_dict[key] = regionsLoader.chip_loader(region_chrom, path, resolution=10000)

test_inputs2, test_targets2 = build_set(hic_generator, chip_generator_dict, batchsize=len(region_chrom))

In [ ]:
test_predictions2 = model.predict(test_inputs2)

In [ ]:
plot_predictions(test_targets2, test_predictions2)

In [ ]:
# target_regions_df = bf.select(all_regions_df, 'chr19').sample(frac=1, random_state=28) #select a chromosome and shuffle it
# hic_generator = hic_loader(target_regions_df, cool_path, resolution=50000)
# chip_generator_dict = {}
# for key, value in bigwig_dict.items():
#     chip_generator_dict[key] = chip_loader(target_regions_df, value, resolution=10000)

# test_inputs2, test_targets2 = build_set(hic_generator, chip_generator_dict, batchsize=4)
# test_predictions2 = model.predict(test_inputs)

In [ ]:
# fig = plt.figure(figsize= (16, 16))
# for i in range(4):
#     ax = plt.subplot(4,4, (i*2)+1)
#     ax.matshow(np.log(test_targets2[i]+5e-6), cmap='YlOrRd')
#     ax.set_title('target')
#     ax = plt.subplot(4,4, (i*2)+2)
#     ax.set_title('prediction')
#     ax.matshow(test_predictions2[i], cmap='YlOrRd')

In [ ]:
# pred_mat = np.exp(model.predict(inputs[0:1])[0])-5e-6

In [ ]:
# plt.imshow(np.log(targets[0]+5e-6), cmap='YlOrRd')

In [ ]:
# plt.imshow(np.log(pred_mat+5e-6), cmap='YlOrRd')

In [ ]:
# P1 = np.array([np.diag(targets[0], i).mean() for i in range(80)])
# P2 = np.array([np.diag(pred_mat, i).mean() for i in range(80)])
# x = np.arange(0, 80)

In [ ]:
# plt.plot(np.log(x[1:]), np.log(P1[1:]))

In [ ]:
# plt.plot(np.log(x[1:]), np.log(P2[1:]))